
---

# 🗃️ FastAPI CRUD – No Pydantic (using SQLite)

A simple FastAPI app that performs basic CRUD operations using SQLite, without using Pydantic models.

---

## 🚀 How to Run

```bash
uvicorn crud:app --reload
```

---

## ⚙️ Database Setup

### 🔧 Connect to SQLite

```python
def get_db():
    conn = sqlite3.connect("employee.db")
    conn.row_factory = sqlite3.Row
    return conn
```

### 🏗️ Create Table

Auto-creates the table if not already present.

```python
with get_db() as db:
    db.execute("""
        CREATE TABLE IF NOT EXISTS employee (
            id INTEGER PRIMARY KEY,
            name TEXT,
            age INTEGER,
            city TEXT
        )
    """)
```

---

## 📄 Endpoints Summary

| Method | Endpoint                         | Description                   |
| ------ | -------------------------------- | ----------------------------- |
| GET    | `/`                              | Home / landing page           |
| POST   | `/create`                        | Create new employee record    |
| GET    | `/employee/view`                 | View all employee records     |
| GET    | `/employee/view/{employee_id}`   | View a single employee record |
| PUT    | `/employee/update/{employee_id}` | Update an existing employee   |
| DELETE | `/employee/delete/{employee_id}` | Delete an employee            |

---

## 🔍 API Endpoints – With Code

### 🏠 1. Home Page

```python
@app.get("/")
def home_page():
    return {"home": "Welcome to landing page of FastAPI"}
```

---

### ➕ 2. Create Employee

```python
@app.post("/create")
def insert_record(name: str, age: int, city: str):
    with get_db() as db:
        db.execute("INSERT INTO employee (name, age, city) VALUES (?, ?, ?)", (name, age, city))
        db.commit()
    return {"message": "employee records inserted successfully"}
```

---

### 📋 3. View All Employees

```python
@app.get("/employee/view")
def view_records():
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee")
        employees = cursor.fetchall()
    return [
        {"id": emp["id"], "name": emp["name"], "age": emp["age"], "city": emp["city"]}
        for emp in employees
    ]
```

---

### 👁️ 4. View Single Employee by ID

```python
@app.get("/employee/view/{employee_id}")
def single_view_record(employee_id: int):
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee WHERE id = ?", (employee_id,))
        employee = cursor.fetchone()
        if employee is None:
            raise HTTPException(status_code=404, detail="Employee not found")
    return {"id": employee["id"], "name": employee["name"], "age": employee["age"], "city": employee["city"]}
```

---

### 🔄 5. Update Employee

```python
@app.put("/employee/update/{employee_id}")
def single_update_record(employee_id: int, name: str, age: int, city: str):
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee WHERE id = ?", (employee_id,))
        if cursor.fetchone() is None:
            raise HTTPException(status_code=404, detail="Employee not found")
        db.execute(
            "UPDATE employee SET name = ?, age = ?, city = ? WHERE id = ?",
            (name, age, city, employee_id)
        )
        db.commit()
    return {
        "message": f"Employee id {employee_id} record updated successfully",
        "id": employee_id, "name": name, "age": age, "city": city
    }
```

---

### ❌ 6. Delete Employee

```python
@app.delete("/employee/delete/{employee_id}")
def single_record_delete(employee_id: int):
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee WHERE id = ?", (employee_id,))
        if cursor.fetchone() is None:
            raise HTTPException(status_code=404, detail="Employee not found")
        db.execute("DELETE FROM employee WHERE id = ?", (employee_id,))
        db.commit()
    return {"message": f"Employee id {employee_id} record deleted successfully"}
```

---